<a href="https://colab.research.google.com/github/winifred-kay/Sentiment-Analysis-with-huggingface-fac/blob/master/ROBERTA_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with Hugging Face

Hugging Face is an open-source and platform provider of machine learning technologies. You can use install their package to access some interesting pre-built models to use them directly or to fine-tune (retrain it on your dataset leveraging the prior knowledge coming with the first training), then host your trained models on the platform, so that you may use them later on other devices and apps.

Please, [go to the website and sign-in](https://huggingface.co/) to access all the features of the platform.

[Read more about Text classification with Hugging Face](https://huggingface.co/tasks/text-classification)

The Hugging face models are Deep Learning based, so will need a lot of computational GPU power to train them. Please use [Colab](https://colab.research.google.com/) to do it, or your other GPU cloud provider, or a local machine having NVIDIA GPU.

In [1]:
!pip install huggingface_hub transformers datasets gradio pipreqs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [3]:
# Import libraries
import os
import pandas as pd
import numpy as np
from scipy.special import softmax
import gradio as gr

from google.colab import drive
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    TFAutoModelForSequenceClassification,
    IntervalStrategy,
    TrainingArguments,
    EarlyStoppingCallback,
    pipeline,
    Trainer
) 


In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Application of Hugging Face Text classification model Fune-tuning

Find below a simple example, with just `3 epochs of fine-tuning`. 

Read more about the fine-tuning concept : [here](https://deeplizard.com/learn/video/5T-iXNNiwIs#:~:text=Fine%2Dtuning%20is%20a%20way,perform%20a%20second%20similar%20task.)

The datasets package is a Python library that provides a collection of over 100 natural language processing (NLP) datasets commonly used for research and development. The library is designed to provide easy access to these datasets, as well as a uniform interface for loading, preprocessing, and working with the data.

The datasets include a range of tasks such as text classification, question answering, named entity recognition, and sentiment analysis, and cover a variety of languages including English, Spanish, French, Chinese, and many others. Some of the popular datasets included in the package are IMDB, COCO, SQuAD, Multi30k, Wikipedia, and Amazon Reviews.

The datasets package is developed by Hugging Face, a company that specializes in NLP and provides a suite of libraries and tools for working with NLP models.




In [5]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [6]:
# Load the dataset and display some values

# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df = pd.read_csv(url)


Data Quality checks 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [8]:
# Select rows with missing values
df.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [9]:
# Select rows with missing values
df[df.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [10]:
# Extract complete text from 'safe_text' column
complete_text = df.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [11]:
# Select row by index and assign values to columns
df.loc[4798, 'label'] = 0
df.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[4798, df.columns.get_loc('safe_text')] = complete_text


In [12]:
df.iloc[4798]

tweet_id             RQMQ0L2A
safe_text    #lawandorderSVU 
label                     0.0
agreement            0.666667
Name: 4798, dtype: object

In [13]:
# Select row by index and assign values to columns
row_index = 4799
df.loc[row_index, 'tweet_id'] = 'RQFQ0G34'
df.loc[row_index, 'label'] = 1
df.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df.iloc[row_index, df.columns.get_loc('safe_text')] = df.iloc[row_index, 1]


In [14]:
df.iloc[4799]

tweet_id     RQFQ0G34
safe_text           1
label             1.0
agreement    0.666667
Name: 4799, dtype: object

In [15]:
df[df.duplicated()].sum()

tweet_id     0.0
safe_text    0.0
label        0.0
agreement    0.0
dtype: float64

I manually split the training set to have a training subset ( a dataset the model will learn on), and an evaluation subset ( a dataset the model with use to compute metric scores to help use to avoid some training problems like [the overfitting](https://www.ibm.com/cloud/learn/overfitting) one ). 

There are multiple ways to do split the dataset. You'll see two commented line showing you another one.

In [16]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [17]:
train.head()

,tweet_id,safe_text,label,agreement
1641,CQDD6QLM,"New <user> ""Hey Love"" #MMR #ManyMenRecords #Yo...",0.0,1.000000
3907,5GV8NEZS,S1256 [NEW] Extends exemption from charitable ...,0.0,1.000000
336,I4D043ST,<user> esp when mercury free vaccines are avai...,1.0,0.666667
6861,CKX52Y8G,"My Life, Your Entertainment #YOTC #MMR @ Exoti...",0.0,1.000000
720,07S3NL2T,Baby Luna is sore from her vaccines :( #poorpuppy,0.0,0.666667


In [18]:
eval.head()

,tweet_id,safe_text,label,agreement
5818,Y8PQ0BT7,So nervous... The baby's getting vaccines... (...,1.0,0.666667
7842,C9Z6JBSS,AIDS N : A malaria vaccine in children with HI...,0.0,0.666667
880,0VE4NWWQ,Measles Outbreak Hits Texas Church That Preach...,1.0,0.666667
9072,RHQRUF14,Thank you <user> for mtg with your staff. We l...,1.0,1.000000
288,ZWEP2IL4,Health district offers no-cost immunizations f...,1.0,0.666667


In [19]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (8000, 4), eval is (2001, 4)


By saving the subsets as CSV files, you can easily load them into your machine learning framework of choice (e.g., PyTorch, TensorFlow) and preprocess the data as needed for your specific task. Additionally, saving the subsets as separate files allows you to easily swap in new training or evaluation data as needed during the development process.

In [20]:
# Save splitted subsets

# Define file path

file_path = "/content/drive/MyDrive/Colab Notebooks/"


train.to_csv(os.path.join(file_path, "train3_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval3_subset.csv"), index=False)

In [21]:
# Load the CSV files into a dataset

from datasets import load_dataset

dataset = load_dataset('csv', data_files={
    'train': file_path + 'train3_subset.csv',
    'eval': file_path + 'eval3_subset.csv'
}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cd0dd7dee0066ce5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Transformers is a Python library for natural language processing (NLP) developed by Hugging Face. It provides an easy-to-use interface for building and training state-of-the-art deep learning models for a variety of NLP tasks, such as text classification, named entity recognition, question answering, and more.

The transformer architecture is a type of neural network that is particularly well-suited for processing sequential data, such as natural language text. It replaces the recurrent neural networks (RNNs) and convolutional neural networks (CNNs) that were previously used for NLP tasks, and has achieved state-of-the-art performance on a wide range of benchmarks.

The Transformers library provides pre-trained transformer models that can be fine-tuned on a specific NLP task with only a small amount of task-specific data. This allows developers to easily leverage the power of transformer models for their own NLP tasks, even if they do not have access to large amounts of training data or high-performance computing resources.

A tokenizer is a component in natural language processing (NLP) that breaks down text into individual tokens, which are usually words or subwords. Tokenization is an important preprocessing step in many NLP tasks, because it converts raw text data into a format that can be easily processed by machine learning models.

There are different types of tokenizers that can be used, depending on the specific requirements of the task. Some common types include:

Word tokenizers: These tokenize text into individual words based on whitespace or punctuation.

Subword tokenizers: These tokenize text into subwords, which can be useful for handling out-of-vocabulary words or words that are rare in the training data.

Character tokenizers: These tokenize text into individual characters, which can be useful for languages that have complex orthographies or for handling misspellings.

AutoTokenizer is used to instantiate a tokenizer. AutoTokenizer is a class in the Transformers library that provides a convenient way to automatically select the appropriate tokenizer for a given pre-trained model. The AutoTokenizer class uses heuristics to determine the type of tokenizer that should be used based on the architecture and configuration of the pre-trained model. This can be useful when working with a variety of pre-trained models, because it allows you to use the appropriate tokenizer without having to manually select one for each model.

In [22]:
tokenizer_roberta = AutoTokenizer.from_pretrained('roberta-base')

Specifically, AutoTokenizer.from_pretrained() is a method in the Transformers library that allows you to load a pre-trained tokenizer for a specific model architecture and configuration. In this case, the from_pretrained() method is called with the argument 'bert-base-cased', which is the name of a pre-trained BERT model that has been trained on a large corpus of English text.

The bert-base-cased configuration refers to a version of the BERT model that has a cased vocabulary, meaning that it distinguishes between uppercase and lowercase letters. This can be useful in tasks where the case of words is important, such as named entity recognition or sentiment analysis.

By instantiating a tokenizer for the bert-base-cased model using AutoTokenizer.from_pretrained(), you can tokenize text according to the same scheme used during pre-training of the BERT model. This can be useful when fine-tuning the pre-trained model on a specific task, because it ensures that the input data is pre-processed in the same way as the data used to train the original model.

In [23]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}


# Define a function to tokenize the text data
def tokenize_data2(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_roberta(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_roberta = dataset_out.map(tokenize_data2, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'
dataset_roberta = dataset_roberta.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

In [24]:
dataset_roberta

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2001
    })
})

In [25]:
train_dataset_roberta = dataset_roberta['train'].shuffle(seed=15)
eval_dataset_roberta = dataset_roberta['eval'].shuffle(seed=15)

In [26]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory where the model checkpoints and evaluation results will be stored
    evaluation_strategy=IntervalStrategy.STEPS,      # Interval for evaluating the model during training (every specified number of steps)
    save_strategy=IntervalStrategy.STEPS,            # Interval for saving the model during training (every specified number of steps)
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Whether to load the best model at the end of training
    num_train_epochs=4,                              # Number of training epochs
    per_device_train_batch_size=4,                   # Batch size per GPU for training
    per_device_eval_batch_size=4,                    # Batch size per GPU for evaluation
    learning_rate=3e-5,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=500,                               # Number of steps between two logs
    fp16=True,                                       # Whether to use 16-bit precision
    gradient_accumulation_steps=16,                  # Number of steps to accumulate gradients before performing an optimizer step
    dataloader_num_workers=2,                        # Number of workers to use for loading data
    push_to_hub=True,                                # Whether to push the model checkpoints to the Hugging Face hub
    hub_model_id="Winnie-Kay/Finetuned_bert_model",  # Model ID to use when pushing the model to the Hugging Face hub 
)

# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Number of epochs with no improvement before stopping training
    early_stopping_threshold=0.01,                   # Minimum improvement in the metric for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [28]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3) 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}


In [30]:
trainer_roberta = Trainer(
    model=model_roberta, 
    args=training_args, 
    train_dataset=train_dataset_roberta, 
    eval_dataset=eval_dataset_roberta,
    compute_metrics=compute_metrics    # Add this line to define the compute_metrics function
)


Cloning https://huggingface.co/Winnie-Kay/Finetuned_bert_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.37k/255M [00:00<?, ?B/s]

Download file runs/May10_03-44-46_1b89ff2b7c60/events.out.tfevents.1683691068.1b89ff2b7c60.6308.2: 100%|######…

Download file runs/May10_03-44-46_1b89ff2b7c60/events.out.tfevents.1683690390.1b89ff2b7c60.6308.0: 100%|######…

Clean file runs/May10_03-44-46_1b89ff2b7c60/events.out.tfevents.1683691068.1b89ff2b7c60.6308.2: 100%|#########…

...

Download file runs/May10_03-44-46_1b89ff2b7c60/1683690390.4340026/events.out.tfevents.1683690390.1b89ff2b7c60.…

Clean file runs/May10_03-44-46_1b89ff2b7c60/1683690390.4340026/events.out.tfevents.1683690390.1b89ff2b7c60.630…

Download file training_args.bin: 100%|##########| 3.75k/3.75k [00:00<?, ?B/s]

Clean file training_args.bin:  27%|##6       | 1.00k/3.75k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/255M [00:00<?, ?B/s]

In [31]:
trainer_roberta.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Rmse
500,0.642900,0.564351,0.604828


TrainOutput(global_step=500, training_loss=0.6428783569335937, metrics={'train_runtime': 1286.3413, 'train_samples_per_second': 24.877, 'train_steps_per_second': 0.389, 'total_flos': 8419629367296000.0, 'train_loss': 0.6428783569335937, 'epoch': 4.0})

In [32]:
# Evaluate the model
eval_results = trainer_roberta.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "Model": "roberta",
    "Loss": eval_results["eval_loss"],
    "RMSE": eval_results["eval_rmse"],
    "Runtime": eval_results["eval_runtime"],
    "Samples Per Second": eval_results["eval_samples_per_second"],
    "Steps Per Second": eval_results["eval_steps_per_second"],
    "Epoch": eval_results["epoch"]
}

# Create a pandas DataFrame from the dictionary
results_df = pd.DataFrame([results_dict])

# Print the results
print(results_df)


     Model      Loss      RMSE  Runtime  Samples Per Second  Steps Per Second  \
0  roberta  0.564351  0.604828  28.6245              69.905            17.502   

   Epoch  
0    4.0  


In [40]:

 # Push the final fine-tuned model to the Hugging Face model hub

trainer_roberta.push_to_hub("Winnie-Kay/Sentiment-Analysis-Roberta-bases")


In [41]:
tokenizer_roberta.push_to_hub("Winnie-Kay/Sentiment-Analysis-Roberta-bases")

CommitInfo(commit_url='https://huggingface.co/Winnie-Kay/Sentiment-Analysis-Roberta-bases/commit/805b7571092dc3306812bdc2ef4c83a74444189a', commit_message='Upload tokenizer', commit_description='', oid='805b7571092dc3306812bdc2ef4c83a74444189a', pr_url=None, pr_revision=None, pr_num=None)

In [42]:
model_roberta.push_to_hub("Winnie-Kay/Sentiment-Analysis-Roberta-bases")

CommitInfo(commit_url='https://huggingface.co/Winnie-Kay/Sentiment-Analysis-Roberta-bases/commit/f97e84d69e15d8d3c7ad9bd198d4994cdce473d7', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='f97e84d69e15d8d3c7ad9bd198d4994cdce473d7', pr_url=None, pr_revision=None, pr_num=None)

### You can load your model from anywhere using from_pretrained!

In [43]:
# Load the tokenizer
tokenizer = tokenizer_roberta.from_pretrained("Winnie-Kay/Sentiment-Analysis-Roberta-bases")

# Load the fine-tuned model
model = pipeline("text-classification", model="Winnie-Kay/Sentiment-Analysis-Roberta-bases", tokenizer=tokenizer)



In [44]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Make predictions on some example text
result = model("I hate COVID19 vaccines.")

# Map the numerical label to the corresponding class name
result[0]["label"] = label_map[int(result[0]["label"].split("_")[1])]

# Print the predicted label and score
print(result)

[{'label': 'neutral', 'score': 0.46742358803749084}]


You may also upload the model on the Hugging Face Platform... [Read more](https://huggingface.co/docs/hub/models-uploading)

This notebook is inspired by an article: [Fine-Tuning Bert for Tweets Classification ft. Hugging Face](https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf)

Do not hesitaite to read more and to ask questions, the Learning is a lifelong activity.

In [47]:
# Define the model path where the pre-trained model is saved on the Hugging Face model hub
model_path = "Winnie-Kay/Sentiment-Analysis-Roberta-bases"

# Initialize the tokenizer for the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the configuration for the pre-trained model
config = AutoConfig.from_pretrained(model_path)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Define a function to preprocess the text data
def preprocess(text):
    new_text = []
    # Replace user mentions with '@user'
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        # Replace links with 'http'
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    # Join the preprocessed text
    return " ".join(new_text)

# Define a function to perform sentiment analysis on the input text
def sentiment_analysis(text):
    # Preprocess the input text
    text = preprocess(text)

    # Tokenize the input text using the pre-trained tokenizer
    encoded_input = tokenizer(text, return_tensors='pt')
    
    # Feed the tokenized input to the pre-trained model and obtain output
    output = model(**encoded_input)
    
    # Obtain the prediction scores for the output
    scores_ = output[0][0].detach().numpy()
    
    # Apply softmax activation function to obtain probability distribution over the labels
    scores_ = softmax(scores_)
    
    # Format the output dictionary with the predicted scores
    labels = ['Negative', 'Neutral', 'Positive']
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }
    
    # Return the scores
    return scores

# Define a Gradio interface to interact with the model
demo = gr.Interface(
    fn=sentiment_analysis, # Function to perform sentiment analysis
    inputs=gr.Textbox(placeholder="Let's hear your thoughts on the covid vaccines..."), # Text input field
    outputs="label", # Output type (here, we only display the label with the highest score)
    interpretation="default", # Interpretation mode
    examples=[["Express your setiments!"]]) # Example input(s) to display on the interface

# Launch the Gradio interface
demo.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>